# Extrem value analysis Tenerife precipitation

This script analyzes **spatial and temporal precipitation data** using **daily precipitation records**, focusing on **extreme value events**.

In [ ]:
#Install some libaries first!
!pip install xclim
!pip install lmoments3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.8/444.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.2/194.2 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.8/69.8 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 2.6 MB/s eta 0:00:00


In [ ]:
#Import need libaries
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as stats

import lmoments3
from lmoments3 import distr

from xclim.core.missing import missing_pct
from xclim.indices.generic import select_resample_op
from xclim.indices.stats import fa, fit, frequency_analysis, parametric_quantile
import gdown

#Download data

We utilize precipitation data from [CHELSA](https://chelsa-climate.org/), a high-resolution climate dataset derived from downscaled ERA5 reanalysis data. A preprocessed dataset is available for download from Google Drive:

In [ ]:
# Download CHELSA precipitation data from Google Drive
#'GOOGLE_DRIVE_LINK' to NetCDF file
url = 'https://drive.google.com/file/d/1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd/view?usp=drive_link'
# Download the NetCDF file
file_path = 'CHELSA_precipitation.nc'
gdown.download(url, file_path, quiet=False, fuzzy=True)

Downloading...
From (original): https://drive.google.com/uc?id=1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd
From (redirected): https://drive.google.com/uc?id=1Z9-YDydbsxFBq4Do43KaB3-Lyzm9dkpd&confirm=t&uuid=b234f8a5-3923-4788-a30c-64b6e9d11223
To: /content/CHELSA_precipitation.nc
100%|██████████| 325M/325M [00:10<00:00, 32.1MB/s]


'CHELSA_precipitation.nc'

#Load data

Since this dataset contains spatial and temporal data, forming a 3D data cube, we will use [XArray](https://docs.xarray.dev/en/stable/), a specialized data protocol designed for handling multi-dimensional scientific data efficiently. To begin, we will open the NetCDF file using XArray:

In [ ]:
#Load precipitation data as XArray
daily_precipitation = xr.open_dataset(file_path)
daily_precipitation

In [ ]:
# create a array which only conains the years - we need this later
years = daily_precipitation['time'].dt.year.values

#Generate precipitation maps

Next, we will generate, plot, and save maps displaying the mean total annual precipitation and the mean annual variance of precipitation over the given time period.

In [ ]:
# Calcualte mean total annual precipitation
# first group by year and sum up values
annual_total_precipitation = daily_precipitation.groupby('time.year').sum(dim='time')
# second calaculate the mean
annual_total_precipitation_mean = annual_total_precipitation.mean(dim='year')

In [ ]:
# Plot the map of mean total annual precipitation
annual_total_precipitation_mean['tp'].plot.imshow()
plt.title('Mean Total Annual Precipitation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Calcualte mean annual variance of precipitation
annual_var_precipitation = daily_precipitation.groupby('time.year').var(dim='time')
annual_var_precipitation_mean = annual_var_precipitation.mean(dim='year')

In [ ]:
# Plot the map of mean annual variance precipitation
annual_var_precipitation_mean['tp'].plot.imshow()
plt.title('Mean Annual Variance of Precipitation')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.show()

In [ ]:
# Store the mean and variance maps as netcdf file
filename = 'CHELSA_Tenerife_mean_total_precipitation_'  + str(years[0]) + '_' + str(years[-1]) + '.nc'
annual_total_precipitation_mean.to_netcdf(filename)
filename = 'CHELSA_Tenerife_mean_variance_precipitation_'  + str(years[0]) + '_' + str(years[-1]) + '.nc'
annual_var_precipitation_mean.to_netcdf(filename)

# **Extreme Value Analysis**

In this section, we will perform an **Extreme Value Analysis (EVA)** to assess extreme precipitation events. The following steps will be carried out:

- **Calculate the block maxima** with an **annual frequency** for each pixel.
- **Fit the Generalized Extreme Value (GEV) distribution** to these maxima for each pixel. *(Note: Depending on the selected method, this step may take some time to compute.)*
- **Estimate extreme precipitation events** corresponding to **5-, 15-, 25-, and 50-year return periods**.
- **Generate complete maps** of expected extreme precipitation events using **QGIS**, incorporating the **Tenerife Shapefile** for spatial context.

To accomplish this, we will utilize the [**xclim library**](https://xclim.readthedocs.io/en/stable/notebooks/frequency_analysis.html), which is already imported in the script header. This library provides specialized tools for climate indices and frequency analysis, facilitating our calculations.

In [ ]:
# Use the libary xclim (https://xclim.readthedocs.io/en/stable/notebooks/frequency_analysis.html)
# to optain the Block Maxima with yearly frequency

In [ ]:
# We use the xclim libary to fit GEV function (dist=dist.gev)
# Therefore, we can select from a broad range of fitting methods:
# - Maximum Likelihood algorithm (method="ML") - Default but very sensetive to outliers
# - Method of Moments (method="MM") - not working with our data
# - Maximum Likelihood Estimation (method="MLE") - kind of similar but diffrent to ML
# - Probability Weighted Moments (method="PWM") - More robust to oulierts!
# PWM is well suited for small samples which in our case make it the prefered method.

In [ ]:
# We use the xclim libary to extract the quantile value for a given return period
# as input we need:
# - the previously fitted parameters (params) of the distribution function
# - the quatinetile we are interested in which relates to the probability of excedance (q = 1-P = 1-1/tau)

In [ ]:
# Plotting the map of expected extreme precipitation of a given return period (RP)

# Befor plotting and saving as 2D map we have to select a quatile - other wise it is a 3D data cube

# Plot mean annual probability precipitation events > threshold

In [ ]:
# Save map as netcdf files